<a href="https://colab.research.google.com/github/sayarghoshroy/LexiNER/blob/main/BE_NER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Getting content from a set of URLs

In [2]:
# Uncomment if boilerpipe3 does not exist
# !pip install boilerpipe3
from boilerpipe.extract import Extractor
from tqdm import tqdm

In [3]:
import nltk
# To Download punkt module:
# nltk.download('punkt')
import pickle
import sys
import json
import re

In [4]:
url_list = ['https://eisamay.indiatimes.com/nation/let-commander-3-other-terrorists-killed-in-kashmir/articleshow/78599215.cms',
            'https://eisamay.indiatimes.com/nation/india-records-74383-covid-19-cases-tally-crosses-70-lakh-mark/articleshow/78599510.cms',
            'https://eisamay.indiatimes.com/nation/delhi-student-18-beaten-to-death-allegedly-over-relationship-with-girl/articleshow/78589618.cms?utm_source=recommended&utm_medium=referral&utm_campaign=article11',
            'https://eisamay.indiatimes.com/entertainment/cinema/sharpshooter-involved-in-2000-attack-on-rakesh-roshan-arrested/articleshow/78593074.cms?utm_source=recommended&utm_medium=referral&utm_campaign=article4',
            'https://eisamay.indiatimes.com/crime/two-men-allegedly-stabbed-tea-shop-worker-after-he-refused-to-give-them-a-cigarette-in-delhi/articleshow/78592505.cms?utm_source=recommended&utm_medium=referral&utm_campaign=article8',
            'https://eisamay.indiatimes.com/entertainment/cinema/corona-positive-bengali-legend-actor-soumitra-chatterjee-is-well-now/articleshow/78587347.cms?utm_source=recommended&utm_medium=referral&utm_campaign=article7',
            'https://eisamay.indiatimes.com/nation/kedarnath-helicopter-service-starts-890-use-chopper-on-first-day/articleshow/78590624.cms?utm_source=recommended&utm_medium=referral&utm_campaign=article4',
            'https://eisamay.indiatimes.com/world/5-killed-in-collision-between-plane-microlight-aircraft-in-france/articleshow/78596847.cms?utm_source=recommended&utm_medium=referral&utm_campaign=article11',
            'https://eisamay.indiatimes.com/nation/india-continued-to-report-a-decline-in-the-active-cases-of-the-coronavirus-govt/articleshow/78592214.cms?utm_source=recommended&utm_medium=referral&utm_campaign=article9',
            'https://eisamay.indiatimes.com/nation/ambulance-driver-who-ferried-200-bodies-of-covid-patients-since-march-dies-of-virus-in-delhi/articleshow/78601226.cms?utm_source=recommended&utm_medium=referral&utm_campaign=article4',
            'https://eisamay.indiatimes.com/west-bengal-news/kolkata-news/wbtc-will-starts-tram-facility-for-pandle-hopping-in-durga-puja-2020/articleshow/78604463.cms?utm_source=recommended&utm_medium=referral&utm_campaign=article4',
            'https://eisamay.indiatimes.com/entertainment/cinema/happy-birthday-amitabh-bachchan-ajay-devgn-to-ayushmann-khurrana-celebs-wish-the-legend/articleshow/78602599.cms?utm_source=recommended&utm_medium=referral&utm_campaign=article1',
            'https://eisamay.indiatimes.com/nation/uttarakhand-queen-of-the-night-brahmakamal-blooming-in-off-season-stumps-many/articleshow/78603528.cms?utm_source=recommended&utm_medium=referral&utm_campaign=article7',
            'https://eisamay.indiatimes.com/west-bengal-news/kolkata-news/people-engaged-in-puja-shopping-during-corona-pandemic/articleshow/78605765.cms?utm_source=recommended&utm_medium=referral&utm_campaign=article11',
            'https://eisamay.indiatimes.com/nation/pm-narendra-modi-says-property-cards-scheme-historical-move-to-transform-rural-india/articleshow/78604053.cms?utm_source=recommended&utm_medium=referral&utm_campaign=article12']

with open('news_urls.json', 'w+') as f:
  json.dump(url_list, f)

with open('news_urls.json', 'r+') as f:
  url_list = json.load(f)

url_list_wiki = ['https://bn.wikipedia.org/wiki/%E0%A6%AD%E0%A6%BE%E0%A6%B0%E0%A6%A4',
                 'https://bn.wikipedia.org/wiki/%E0%A6%AD%E0%A6%BE%E0%A6%B0%E0%A6%A4%E0%A7%87%E0%A6%B0_%E0%A6%AA%E0%A7%8D%E0%A6%B0%E0%A6%BE%E0%A6%A3%E0%A7%80',
                 'https://bn.wikipedia.org/wiki/%E0%A6%95%E0%A6%B2%E0%A6%95%E0%A6%BE%E0%A6%A4%E0%A6%BE',
                 'https://bn.wikipedia.org/wiki/%E0%A6%A8%E0%A7%80%E0%A6%B2%E0%A6%97%E0%A6%BF%E0%A6%B0%E0%A6%BF_%E0%A6%B2%E0%A7%87%E0%A6%99%E0%A7%8D%E0%A6%97%E0%A7%81%E0%A6%B0',
                 'https://bn.wikipedia.org/wiki/%E0%A6%A4%E0%A7%87%E0%A6%B2%E0%A7%87%E0%A6%99%E0%A7%8D%E0%A6%97%E0%A6%BE%E0%A6%A8%E0%A6%BE',
                 'https://bn.wikipedia.org/wiki/%E0%A6%86%E0%A6%B8%E0%A6%BE%E0%A6%AE',
                 'https://bn.wikipedia.org/wiki/%E0%A6%B9%E0%A6%B0%E0%A6%BF%E0%A6%AF%E0%A6%BC%E0%A6%BE%E0%A6%A8%E0%A6%BE',
                 'https://bn.wikipedia.org/wiki/%E0%A6%B9%E0%A6%BF%E0%A6%AE%E0%A6%BE%E0%A6%9A%E0%A6%B2_%E0%A6%AA%E0%A7%8D%E0%A6%B0%E0%A6%A6%E0%A7%87%E0%A6%B6',
                 'https://bn.wikipedia.org/wiki/%E0%A6%95%E0%A6%B0%E0%A7%8D%E0%A6%A3%E0%A6%BE%E0%A6%9F%E0%A6%95',
                 'https://bn.wikipedia.org/wiki/%E0%A6%AE%E0%A6%BE%E0%A6%B2%E0%A6%AF%E0%A6%BC%E0%A6%BE%E0%A6%B2%E0%A6%AE_%E0%A6%AD%E0%A6%BE%E0%A6%B7%E0%A6%BE',
                 'https://bn.wikipedia.org/wiki/%E0%A6%AA%E0%A6%B0%E0%A7%8D%E0%A6%A4%E0%A7%81%E0%A6%97%E0%A6%BE%E0%A6%B2',
                 'https://bn.wikipedia.org/wiki/%E0%A6%A8%E0%A7%87%E0%A6%A6%E0%A6%BE%E0%A6%B0%E0%A6%B2%E0%A7%8D%E0%A6%AF%E0%A6%BE%E0%A6%A8%E0%A7%8D%E0%A6%A1%E0%A6%B8',
                 'https://bn.wikipedia.org/wiki/%E0%A6%AB%E0%A7%8D%E0%A6%B0%E0%A6%BE%E0%A6%A8%E0%A7%8D%E0%A6%B8',
                 'https://bn.wikipedia.org/wiki/%E0%A6%AF%E0%A7%81%E0%A6%95%E0%A7%8D%E0%A6%A4%E0%A6%B0%E0%A6%BE%E0%A6%9C%E0%A7%8D%E0%A6%AF',
                 'https://bn.wikipedia.org/wiki/%E0%A6%9C%E0%A6%AE%E0%A7%8D%E0%A6%AE%E0%A7%81_%E0%A6%93_%E0%A6%95%E0%A6%BE%E0%A6%B6%E0%A7%8D%E0%A6%AE%E0%A7%80%E0%A6%B0',
                 'https://bn.wikipedia.org/wiki/%E0%A6%B8%E0%A7%81%E0%A6%87%E0%A6%9C%E0%A6%BE%E0%A6%B0%E0%A6%B2%E0%A7%8D%E0%A6%AF%E0%A6%BE%E0%A6%A8%E0%A7%8D%E0%A6%A1',
                 'https://bn.wikipedia.org/wiki/%E0%A6%87%E0%A6%A4%E0%A6%BE%E0%A6%B2%E0%A6%BF',
                 'https://bn.wikipedia.org/wiki/%E0%A6%85%E0%A6%B8%E0%A7%8D%E0%A6%9F%E0%A7%8D%E0%A6%B0%E0%A6%BF%E0%A6%AF%E0%A6%BC%E0%A6%BE',
                 'https://bn.wikipedia.org/wiki/%E0%A6%9C%E0%A6%BE%E0%A6%B0%E0%A7%8D%E0%A6%AE%E0%A6%BE%E0%A6%A8%E0%A6%BF',
                 'https://bn.wikipedia.org/wiki/%E0%A6%AE%E0%A6%BE%E0%A6%A6%E0%A7%8D%E0%A6%B0%E0%A6%BF%E0%A6%A6']

with open('wiki_urls.json', 'w+') as f:
  json.dump(url_list_wiki, f)

with open('wiki_urls.json', 'r+') as f:
  url_list_wiki = json.load(f)

print("Number of News URLs: " + str(len(url_list)))
print("Number of Wikipedia URLs: " + str(len(url_list_wiki)))

Number of News URLs: 15
Number of Wikipedia URLs: 20


In [5]:
content_collection = []
# content_collection format
# {'url': <article_URL>, 'content': <boilerpipe_content>}

failed_extraction_count = 0
failed_URLs = []

for idx in tqdm(range(len(url_list_wiki))):
  url = url_list_wiki[idx]
  try:
    extractor = Extractor(extractor = 'ArticleExtractor', url = url)
    content = str(extractor.getText())
    # extractor.getText() returns a java lang string
    content_collection.append({'url': url, 'content': content})
  except Exception as e:
    failed_extraction_count += 1
    failed_URLs.append(url)
    print("Error Extracting Page at " + url)

print("")
print("Number of URL Contents Extracted Successfully = " + str(len(url_list_wiki) - failed_extraction_count))
print("Set of Failed URLs: ")
print(failed_URLs)

with open('working_content.json', 'w+') as f:
  json.dump(content_collection, f)

100%|██████████| 20/20 [00:07<00:00,  2.62it/s]


Number of URL Contents Extracted Successfully = 20
Set of Failed URLs: 
[]


#### Framing Named Entity Recognition Rules:

- Refer to this [resource for Bengali NER](http://cdn.iiit.ac.in/cdn/ltrc.iiit.ac.in/ner-ssea-08/drafts/1.pdf)


In [6]:
def clean(txt):
  # Cleans Text for Custom Tokenization
  # Removes English Characters and Numbers

  txt = txt.replace("\n", " ").replace("\r", " ").replace("|", " | ").replace("।", " । ")
  # In Bengali, । marks the end of a sentence
  punc_list = '!"#$&*+,.-/;?@\^_~)(][}{%'
  # Removing a set of special characters
  t = str.maketrans(dict.fromkeys(punc_list, " "))
  txt = txt.translate(t)
  # Removing single quotes and backticks
  t = str.maketrans(dict.fromkeys("'`", ""))
  txt = txt.translate(t)
  return txt

def regtok(txt):
  txt = clean(txt)
  regex = re.compile(r'(\d+|\s+|=|}}|\|)')
  tokens = [token.strip() for token in regex.split(txt) if token.strip() != '' and token.strip().isalnum() != True]
  return tokens

In [7]:
loc_token_end = ['সরনী', 'রোড', 'স্ট্রিট', 'লেন', 'থানা', 'স্কুল', 'বিদ্যালয়', 'কলেজ', 'নদী', 'লেক', 'হ্রদ', 'সাগর', 'মহাসাগর', 'পাহাড়', 'পর্বত']
loc_connect_end = ['নগর', 'গঞ্জ', 'গ্রাম', 'পুর', 'গড়']
person_begin = ['শ্রী', 'শ্রীযুক্ত', 'ডঃ', 'মিঃ', 'মিস', 'মিসেস', 'বেগম', 'বিবি', 'মোঃ', 'ডাক্তার', 'ডক্টর', 'স্বামী', 'সৈয়দ', 'রেভারেন্ড', 'প্রফ', 'অধ্যাপক', 'প্রধানমন্ত্রীর'] 
person_end = ['বাবু', 'দাদা', 'দা', 'সাহেব', 'কাকু']
surnames = ['ঘোষ', 'বোস', 'বসু', 'মিত্র', 'রায়', 'সরকার', 'খান', 'আহমেদ', 'রহমান', 'হক']
person_end += surnames
person_ref_end = ['রা', 'এরা', 'দের', 'কে']
person_marker = ['ৎ', ' ँ']
ltd_prev_end = ['পাবলিক', 'প্রাইভেট']
limited = 'লিমিটেড'
and_name = 'এণ্ড'
and_end = ['সনস', 'সন', 'ডটার', 'ডটারস', 'ডটার্স ', 'কম্পানি', 'কোম্পানি']
org_end = ['ইনস্টিটিউট', 'এন্টারপ্রাইস', 'কোম্পানি', 'কোম্পানি', 'কর্পোরেশন']

def get_named_entities(text):
  text = clean(text)
  tokens = regtok(text)
  size = len(tokens)
  
  tags = ['none' for idx in range(size)]
  # default tag is 'none'

  left = 0
  # marks the beginning of a possible arconym

  for idx in range(size):
    token = tokens[idx]
    if token in loc_connect_end:
      if idx > 0 and tokens[idx - 1] != '|' and tokens[idx - 1] != '।':
        tags[idx - 1] = 'loc'
        tags[idx] = 'loc'

    if idx > 1 and token == limited and tokens[idx - 1] in ltd_prev_end:
      tags[idx - 2] = 'org'
      tags[idx - 1] = 'org'
      tags[idx] = 'org'
      print("Organization Found")

    if idx > 0 and idx < (size - 1) and token == and_name and tokens[idx + 1] in and_end:
      tags[idx - 1] = 'org'
      tags[idx] = 'org'
      tags[idx + 1] = 'org'
      print("Organization Found")

    if token in org_end:
      if idx > 0 and tokens[idx - 1] != '|' and tokens[idx - 1] != '।':
        tags[idx - 1] = 'org'
        tags[idx] = 'org'
        print("Organization Found")

    for init in loc_token_end:
      if token.endswith(init):
        tags[idx] = 'loc'

    for init in person_begin:
      if token.startswith(init):
        tags[idx] = 'per'
        if idx + 1 > size and tokens[idx + 1] != '|' and tokens[idx + 1] != '।':
          tags[idx + 1] = 'per'

    for init in person_end:
      if token.endswith(init):
        tags[idx] = 'per'

    for init in person_ref_end:
      if token.endswith(init):
        tags[idx] = 'per'

    for init in person_marker:
      if init in token:
        tags[idx] = 'per'

    if len(token) <= 2 and token != '|' and token != '।':
      pass
    
    elif len(token) > 2 or token == '|' or token == '।':
      if idx - left <= 2:
        left = idx + 1
      else:
        print("Organization Acronym Found")
        while left != idx:
          print(tokens[left])
          tags[left] = 'org'
          left += 1
        left = idx + 1
  
  return tags

for page in content_collection:
  get_named_entities(str(page['content']))

Organization Found
Organization Found
Organization Found
Organization Found
Organization Found


In [8]:
# ^_^ Thank You